In [1]:
import sys
import os
import time
import json
import csv
import threading
import queue
import asyncio 
import nest_asyncio
nest_asyncio.apply()
import twint

sys.path.insert(1, 'C:/Users/Luca/Aug20_Ditchley')
from src.data import journalists as journos

1. Getting journalist twitter handles according to topic¶

In [2]:
keyword = 'political'
journo_handles = journos.get_handles_by_keyword(keyword)
print(len(journo_handles))

236


In [8]:
len(journo_handles)

234

In [7]:
#journo_handles.remove('Tinglepolitics')# no tweets in the target date period
#journo_handles.remove('P_Madeley_Star') # locked tweets
#journo_handles.remove('LauraHusbo') # no tweets in the target date period
#journo_handles.remove('AndyRoocroft') # locked tweets
#journo_handles.remove('dzuidijk') # no tweets in the target date period
#journo_handles.remove('journomatei') # no tweets in the target date period

ValueError: list.remove(x): x not in list

In [9]:
num_threads = 6 # num_threads can be any amount, but only so many can be run in parallel, according to number of cores you have.
# import multiprocessing; multiprocessing.cpu_count() can tell you how many cores python can see.

In [10]:
### from here I try the same but for tweets

def get_tweets(q, fp, persist):
    '''
    Params
    ------
    q : a Queue instance
    
    fp : string
        Provides the path where the file should be saved.
    persist : bool
        If True, make a repeat attempt if attempt fails.
    '''
    while True: 
        username = q.get()
        success = False
        filepath = fp +username+'.csv'
        while not success:
            print('Attempting to get tweets of @'+username)
            c = twint.Config()
            c.Username = username
            #c.User_full = False
            c.Hide_output = True
            #c.Pandas =True
            #c.Store_object = True
            c.Limit = 10000000
            c.Until = '2020-08-01'
            c.Since = '2019-08-01'
            c.Profile_full = True
            c.Store_csv = True
            c.Retweets = True
            #c.Native_retweets = True
            c.Output = filepath

            twint.run.Search(c) # Profile
            tweets = twint.storage.panda.Tweets_df
            #tweets = tweets.append(twint.storage.panda.Tweets_df)
            
            if persist:
                if os.path.exists(filepath):
                    success = True
                    print('Tweets of @'+username+' saved.')
            else:
                success = True
        q.task_done()

In [ ]:
q = queue.Queue(maxsize=0)

for i in range(num_threads): # Loop to create threads
    worker = threading.Thread(target=get_tweets, args=(q,'../data/raw/'+keyword +'_', True)) # Change persist to True to tell the code to keep trying until results are written to file.
    worker.setDaemon(True)
    worker.start()

for username in journo_handles: # Loop to add journalists usernames to the queue
#for username in ['jennystrasburg',]: 
    q.put(username)

q.join()

Check who failed and who did not

In [11]:
all_handles = []
all_users = []
failed = []
for name in journo_handles:
    filepath = '../data/raw/'+keyword +'_'+name+'.csv'
    if not os.path.exists(filepath):
        failed.append(name)
    if os.path.exists(filepath):
        all_handles.append(filepath)   

In [13]:
failed

['Simon_Vaughan']

In [14]:
#this journalist has a locked account
#failed.remove('Tinglepolitics')# no tweets in the target date period
#failed.remove('P_Madeley_Star') # locked tweets
#failed.remove('LauraHusbo') # no tweets in the target date period
#failed.remove('AndyRoocroft') # locked tweets
#failed.remove('dzuidijk') # no tweets in the target date period
#failed.remove('journomatei') # no tweets in the target date period
#failed.append('mateirosca')
#journo_handles.remove('AndyRoocroft')
failed.remove('Simon_Vaughan') # no tweets in the target date period

Re run the multithread to see if we can scrape some more from the failed list

In [ ]:
q = queue.Queue(maxsize=0)

for i in range(num_threads): # Loop to create threads
    worker = threading.Thread(target=get_tweets, args=(q, '../data/raw/'+keyword+'_', True))
    worker.setDaemon(True)
    worker.start()

for username in failed: # Loop to add journalists usernames to the queue
    q.put(username)

q.join()

collate all the individual csv files together

In [15]:
import pandas as pd
all_handles = []
all_users = []
failed = []
all_tweets = pd.DataFrame()
for name in journo_handles:
    filepath = '../data/raw/'+keyword +'_'+name+'.csv'
    if os.path.exists(filepath):
        all_handles.append(filepath)
        temp_csv = pd.read_csv(filepath)
        all_tweets = pd.concat([all_tweets, temp_csv])

In [16]:
len(all_tweets)

261411

In [17]:
all_tweets.index = range(len(all_tweets))

In [18]:
all_tweets.to_csv('twint_political_19082020_inParallel_with_index.csv', index = True)